In [1]:
# Instalar la librería necesaria
%pip install xlsxwriter

import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
import requests
import io
import pandas as pd
import traceback
from IPython.display import HTML

# Título en HTML
title_html = "<h1 style='font-size:36px; color:blue; text-align:center; font-weight:bold;'>PROGRAMA ETIQUETAS AZTK SPORTS</h1>"
title_widget = widgets.HTML(value=title_html)

# Widget para subir archivo CSV
upload_widget = widgets.FileUpload(accept='.csv', multiple=False)
# Label de estado
status_label = widgets.Label("Esperando a que suba un archivo CSV.")
# Botón para procesar
process_button = widgets.Button(
    description='Procesar Archivo CSV',
    disabled=True,
    button_style='success',
    tooltip='Click para procesar el archivo CSV subido',
    icon='cogs',
    layout=widgets.Layout(width='250px', height='60px')
)
# Área de salida
output_area = widgets.Output()

# Cargar imagen
try:
    response = requests.get("https://http2.mlstatic.com/storage/mshops-appearance-api/images/70/781918470/logo-2022092822515684700.png")
    response.raise_for_status()
    image_widget = widgets.Image(value=response.content, format='png', width=350, height=250)
except requests.exceptions.RequestException as e:
    print(f"Error al cargar la imagen: {e}")
    image_widget = widgets.Label(value="Error al cargar la imagen.")

# Crear un contenedor centrado para los widgets
centered_box = widgets.VBox(
    [
        title_widget,
        image_widget,
        upload_widget,
        process_button,
        status_label,
        output_area
    ],
    layout=widgets.Layout(
        align_items='center',  # Centrar horizontalmente
        width='100%'
    )
)

# Estilo CSS adicional para fondo y centrado++++
html_background = HTML(f"""
<style>
body {{
    background-image: url('https://www.diarioviral.net/wp-content/uploads/2024/01/45835603-fondo-blanco-abstracto-con-líneas-suaves.jpg');
    background-size: cover;
    background-position: center;
    background-repeat: no-repeat;
    background-color: #f0f0f0;
    color: black !important;
}}
* {{
    color: black !important;
}}
/* Opcional: centrar todo el contenido si deseas */
</style>
""")
# Mostrar estilos
display(html_background)

# Mostrar todo en un solo contenedor centrado
display(centered_box)

# Funciones
def handle_upload(change):
    with output_area:
        clear_output()
        if upload_widget.value:
            process_button.disabled = False
            status_label.value = "Archivo subido. Haga clic en 'Procesar Archivo CSV' para continuar."
        else:
            process_button.disabled = True
            status_label.value = "Esperando a que suba un archivo CSV."

def handle_process(b):
    with output_area:
        clear_output()
        if not upload_widget.value:
            status_label.value = "Por favor, suba un archivo CSV primero."
            return
        try:
            uploaded_filename = list(upload_widget.value.keys())[0]
            uploaded_content = upload_widget.value[uploaded_filename]['content']
            status_label.value = f'Procesando archivo "{uploaded_filename}" con tamaño {len(uploaded_content)} bytes...'
            df = pd.read_csv(io.BytesIO(uploaded_content))

            # Renombrar columnas existentes
            rename_map = {
                'Order Deadline': 'orderDate',
                'Order Lines/Product/Brand': 'brand',
                'Order Reference': 'PO',
                'Order Lines/Product/Descripción Aduana': 'partDesc',
                'Order Lines/Product/Tecnología': 'Tecnologia',
                'Order Lines/Product/Name': 'modelo',
                'Order Lines/Product/Internal Reference': 'item',
                'Order Lines/Product/Inner': 'inner',
                'Order Lines/Product/Calibre': 'caliber',
                'Order Lines/Quantity': 'qty',
                'Order Lines/Product/Joules': 'Joules'
            }
            columns_to_rename = {col: new_name for col, new_name in rename_map.items() if col in df.columns}
            df = df.rename(columns=columns_to_rename)
            print("Columnas renombradas si existen.")

            # Modificación para cambiar valores en 'Order Lines/Product/Packagings'
            packaging_col = 'Order Lines/Product/Packagings'
            if packaging_col in df.columns:
                df[packaging_col] = df[packaging_col].replace({
                    'Master PR900SGEN3': '3',
                    'Master PR900GEN2.55': '4',
                    'Master PR900WGEN3': '3',
                    'Master PR900SGEN2.55': '4',
                    'Master T-REX W.55': '2',
                    'Master T-REX .55': '2',
                    'Master T-REX BULLPUP .55': '2',
                    'Master': '2',
                    'Master M30C.55': '2',
                    'Master M16.55': '2',
                    'Master M40.55': '2',
                    'Master P35.55': '2',
                    'Master P35Camo.55': '2',
                    'Master P35X.55': '2',
                    'raptor': '1',
                    'Master P35.55': '2',
                })
                print(f"Valores en '{packaging_col}' actualizados.")
            else:
                print(f"Columna '{packaging_col}' no encontrada en el archivo CSV.")

            # Procesar fecha
            if 'orderDate' in df.columns:
                df['orderDate'] = pd.to_datetime(df['orderDate'], errors='coerce')
                df['orderDate'] = df['orderDate'].dt.strftime('%Y-%m-%d')
                print("Columna 'orderDate' actualizada para mostrar solo la fecha.")

            # Llenar condicional
            if 'brand' in df.columns and 'PO' in df.columns and 'orderDate' in df.columns:
                non_null_brand_indices = df[df['brand'].notnull()].index
                if not non_null_brand_indices.empty:
                    last_brand_index = non_null_brand_indices[-1]
                    df.loc[:last_brand_index, 'orderDate'] = df.loc[:last_brand_index, 'orderDate'].ffill()
                    df.loc[:last_brand_index, 'PO'] = df.loc[:last_brand_index, 'PO'].ffill()
                    print("Llenado condicional aplicado a 'orderDate' y 'PO'.")
                else:
                    print("No se encontraron valores no nulos en 'brand'. No se aplicó el llenado condicional.")
            else:
                print("Una o más de las columnas 'brand', 'PO', 'orderDate' no encontradas para aplicar llenado condicional.")

            # Joules lógica para usar el nuevo nombre de 'Joules'
            if 'Joules' in df.columns:
                null_joules_mask = df['Joules'].isnull()
                if not pd.api.types.is_string_dtype(df['Joules']):
                    df['Joules'] = df['Joules'].astype(str)
                df.loc[null_joules_mask, 'Joules'] = "N/A"
                print("Valores nulos en 'Joules' reemplazados por 'N/A'.")
            else:
                print("Columna 'Joules' no encontrada.")

            status_label.value = "Procesamiento completado. Preparando descarga..."

            # Antes de guardar, renombrar 'Order Lines/Product/Packagings' a 'master'
            if 'Order Lines/Product/Packagings' in df.columns:
                df = df.rename(columns={'Order Lines/Product/Packagings': 'master'})
                print("Columna 'Order Lines/Product/Packagings' renombrada a 'master'.")

            # Guardar en Excel con formato para alinear las columnas 'Joules' y 'master' a la derecha
            output_filename_excel = 'modified_' + list(upload_widget.value.keys())[0].replace('.csv', '.xlsx')
            try:
                # Crear un escritor de Excel con xlsxwriter
                writer = pd.ExcelWriter(output_filename_excel, engine='xlsxwriter')
                df.to_excel(writer, index=False, sheet_name='Sheet1')

                # Obtener el objeto workbook y worksheet
                workbook = writer.book
                worksheet = writer.sheets['Sheet1']

                # Crear formato para alinear a la derecha
                right_align_format = workbook.add_format({'align': 'right'})

                # Identificar la columna 'Joules' (índice de la columna)
                if 'Joules' in df.columns:
                    joules_col_idx = df.columns.get_loc('Joules')
                    # Aplicar formato de alineación a la derecha a toda la columna 'Joules'
                    worksheet.set_column(joules_col_idx, joules_col_idx, None, right_align_format)

                # Identificar la columna 'master' (índice de la columna)
                if 'master' in df.columns:
                    master_col_idx = df.columns.get_loc('master')
                    # Aplicar formato de alineación a la derecha a toda la columna 'master'
                    worksheet.set_column(master_col_idx, master_col_idx, None, right_align_format)

                # Guardar el archivo
                writer.close()
                status_label.value = "Archivo Excel guardado y listo para descargar."
                files.download(output_filename_excel)
            except Exception as e:
                print(f"Error al guardar el archivo Excel: {e}")
                status_label.value = f"Error al guardar el archivo Excel: {e}"

        except Exception as e:
            print("Error durante el procesamiento:")
            traceback.print_exc()

# Enlace de eventos
upload_widget.observe(handle_upload, names='value')
process_button.on_click(handle_process)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 3.6 MB/s eta 0:00:00
